# Stationnary clustering and companies evolution

In this notebook we build stationnary clusters on a given period (from 2010 to 2018). They are meant to be representative of the financial segmentation of the market during the period. The period chosen exclude the 2008 economic crisis since we first want to base our study on a stationnary global economic regime (post-crisis economic growth rebound).

After building clusters representing different long-term (8 years) performances of the companies available on the 2010-2018 period, we observe companies affilliation to clusters with respect to time (year by year). We hope observing pattern such as companies switching from one cluster to another and we would like to give the reasons of those transitions.

## Data loading

In [ ]:
import pandas as pd
import matplotlib
import numpy as np

matplotlib.rcParams['figure.figsize'] = (12.0, 8.0)


path_data = '/home/rho/Documents/work/bnp_re/company_clustering/data/'
path = path_data + 'df_company_quant.pickle'
path_qualitative = path_data + 'df_company_qualitative.pickle'
path_adress = path_data + 'df_company_adress.pickle'


df_company_quant = pd.read_pickle(path)
df_company_qualitative = pd.read_pickle(path_qualitative)
df_company_adress = pd.read_pickle(path_adress)

## DataFrame building

To perform the analysis we have to highlight yearly financial data for all companies.

### Reshape the data to add a year index

In [ ]:
df = (df_company_quant
 .drop(columns=['LAST_STATUS_DATE', 'LAST_FINANCIAL_DATE', 'NB_SHAREHOLDERS',
                'NB_SHAREHOLDINGS', 'ENTITY_START_DATE', ])
 .replace(0, np.nan))

df = (df
 .T
 .set_index([df.T.index.str.split('_').str[-1].set_names('year'),
             df.T.index.str.split('_').str[:-1].set_names('variable')
                                                     .map(lambda x: f'{x[0]}_{x[1]}'
                                                                 if len(x) == 2
                                                                    else x[0])])
 .unstack('variable')
 .stack(0)
 .assign(EBITDA_BY_INCOME=lambda df: df.EBITDA / df.OP_INCOME,
         ASSET_BY_INCOME=lambda df: df.ASSET / df.OP_INCOME,
         RESULT_BY_INCOME=lambda df: df.RESULT / df.OP_INCOME,
         EBITDA_BY_STAFF=lambda df: df.EBITDA / df.STAFF_COST,
         INCOME_BY_STAFF=lambda df: df.OP_INCOME / df.STAFF_COST,
         EMPLOYEE_COST=lambda df: df.NB_EMPLOYEES / df.STAFF_COST,
         )
 
 .drop('2019')
 .drop('NB_EMPLOYEES', axis=1))

df.index = df.index.set_names(['year', 'id'])

In [ ]:
df

### Computing returns by year

In [ ]:
df_returns = (df             
 .unstack(level=1)
 .pct_change()
 .replace(0, np.nan))

df_returns = df_returns.multiply(df.unstack(level=1).shift().apply(np.sign))
df_returns.columns = df_returns.columns.set_names(['variable', 'id'])

### Building DataFrame contaning both returns and absolute values

As requested by Xavier, we mix returns and absolute values to compute clusters.

In [ ]:
df_all =\
(df_returns
 .stack(1)
 .join(df[['EBITDA', 'OP_INCOME', 'STAFF_COST']], lsuffix='_RETURNS')
 .unstack(1))

Below a sample of the DataFrame used to compute clusters.

In [ ]:
(df_all
 .stack(1)
 .sample(frac=1)
 .head(10))

### Aggregating values over the whole period to generate stationnary companies (time independant).

We take the median over the entire time period in order to obtain for each of the variables, a value that represents the global performance of the company over the whole economic period.

In [ ]:
df_final = (df_all
 .stack(level=1)
 .groupby(level=1)
 .median()
#  # To drop before
 .join(df_company_quant[['NB_SHAREHOLDERS', 'NB_SHAREHOLDINGS', 'ENTITY_START_DATE', 'NB_EMPLOYEES_2018']])
 .assign(AGE=lambda df: (2020 - df.ENTITY_START_DATE.dt.year))
 .rename(columns={'NB_EMPLOYEES_2018': 'NB_EMPLOYEES'})
 .drop('ENTITY_START_DATE', axis=1)
 .dropna()
)

df_final

## Compute stationnary clusters

In [ ]:
alpha = 0.01

mask_outliers = (df_final
     .apply(lambda x:
            ((x > x.quantile(1 - alpha))
           | (x < x.quantile(alpha))))
     .any(axis=1))

from sklearn import mixture

gmm = mixture.GaussianMixture(n_components=4)
gmm.fit(df_final.loc[~mask_outliers])

df = (df_final
 .assign(cluster=gmm.predict(df_final))
 .loc[~mask_outliers])

from plotly.subplots import make_subplots
import plotly.graph_objects as go

ncol = 4
nrow = len(df_final.columns) // ncol + 1
variable_names = df.drop(columns='cluster').columns
nb_variables = len(variable_names)
colors = ['red', 'blue', 'orange', 'green']


fig = make_subplots(rows=nrow, cols=ncol, shared_yaxes=False, subplot_titles=variable_names,
                    vertical_spacing=0.05, horizontal_spacing=0.05)

for i, column_name in enumerate(variable_names):
    for j in range(df.cluster.max() + 1):

        col = (i % ncol) + 1
        row = i // (ncol) + 1
        fig.add_trace(go.Box(y=df.loc[df.cluster.eq(j), column_name],
                             marker_color=colors[j],
                             name=j),
                      row=row,
                      col=col)

fig.update_layout(showlegend=False,
                  autosize=False,
                  width=1000,
                  height=1000)

fig.update_layout()
fig.show()
del fig

In [ ]:
df_final = (df_all
 .stack(level=1)
 .groupby(level=1)
 .median()
#  # To drop before
 .join(df_company_quant[['NB_SHAREHOLDERS', 'NB_SHAREHOLDINGS', 'ENTITY_START_DATE', 'NB_EMPLOYEES_2018']])
 .drop(columns=['NB_SHAREHOLDINGS', 'NB_SHAREHOLDERS'])
 .assign(AGE=lambda df: (2020 - df.ENTITY_START_DATE.dt.year))
 .rename(columns={'NB_EMPLOYEES_2018': 'NB_EMPLOYEES'})
 .drop('ENTITY_START_DATE', axis=1)
 .dropna())

alpha = 0.01

mask_outliers = (df_final
     .apply(lambda x:
            ((x > x.quantile(1 - alpha))
           | (x < x.quantile(alpha))))
     .any(axis=1))

from sklearn import mixture

gmm = mixture.GaussianMixture(n_components=4, random_state=94)
gmm.fit(df_final.loc[~mask_outliers])

df = (df_final
 .assign(cluster=gmm.predict(df_final))
 .loc[~mask_outliers])

df.to_pickle(path=path_data + 'df_clustering.pickle', protocol=0)

from plotly.subplots import make_subplots
import plotly.graph_objects as go

ncol = 4
nrow = len(df_final.columns) // ncol + 1
variable_names = df.drop(columns='cluster').columns
nb_variables = len(variable_names)
colors = ['red', 'blue', 'orange', 'green']


fig = make_subplots(rows=nrow, cols=ncol, shared_yaxes=False, subplot_titles=variable_names,
                   vertical_spacing=0.05, horizontal_spacing=0.05)

for i, column_name in enumerate(variable_names):
    for j in range(df.cluster.max() + 1):

        col = (i % ncol) + 1
        row = i // (ncol) + 1
        fig.add_trace(go.Box(y=df.loc[df.cluster.eq(j), column_name],
                             marker_color=colors[j],
                             name=j),
                      row=row,
                      col=col,)

fig.update_layout(showlegend=False,
                  autosize=False,
                  width=1000,
                  height=1000)

fig.update_layout()
fig.show()
del fig

### Cluster description:

- Cluster 2 (yellow) contains companies that have the best growth over the period since it has the highest median in almost all financial variables (ASSET, EBITDA, OP_INCOME, PROFIT_RATIO, ...). However it has also a growing STAFF_COST.


- Cluster 0 and 1 (red and blue) are quite similar, they might represent company with medium performances. Red has a slightly better OP_INCOME and ASSET returns over the 2010-2018 period. And is better to reduce EMPLOYEE_COST. But its STAFF_COST is more increasing than blue cluster. Finally, cluster 0 contains a small number of employees while blue contains a way higher number of employees. Blue is older than red.


- Green represents small companies with bad results.


- Red are small companies that have a quite good performance during the period while green are small companies without great results over the 2010-2018 horizon.

### Assign cluster names

Defining name of clusters here

In [ ]:
cluster_names = ['Young - Slow growth', 'Old - Big companies', 'Old - Slow growth', 'Young - Fast growth']

In [ ]:
(df_final
 .assign(cluster=gmm.predict(df_final),
         cluster_count=lambda df:  df.groupby('cluster').transform('count').iloc[:, 0])
 .replace({'cluster': {i: cluster_names[i] for i in range(len(cluster_names))}})
 .to_pickle(path=path_data + 'df_clustering_graphs.pickle', protocol=0))

### Number of companies by cluster

In [ ]:
(df_final
 .assign(cluster=gmm.predict(df_final))
 .cluster
 .value_counts()
 .plot
 .bar()
 .set(xlabel='Cluster Id',
      ylabel='Number of companies'))

### Remarks

- Cluster 3 contains the most companies.
- Cluster 0 is the second most populated cluster.
- Both clusters 2 and 1 have the same amount of companies.
- Clustering is quite well balanced.

In [ ]:
(df_final
 .assign(cluster=gmm.predict(df_final))
 .cluster
 .value_counts()
 .divide(len(df_final.index))
 .plot
 .bar()
 .set(xlabel='Cluster Id',
      ylabel='Rate of companies'))

### Remarks

- Cluster 3 contains the most companies.
- Cluster 0 is the second most populated cluster.
- Both clusters 2 and 1 have the same amount of companies.

### Activity type with respect to clusters.

In [ ]:
(df_final
 .assign(cluster=gmm.predict(df_final))
 
 .join(df_company_qualitative)
 .groupby(['cluster', 'FIRST_ACTIVITY_TYPE_ID'])
 .count()
 .ASSET
 .unstack(0)
 .dropna()
 .loc[lambda x: x.min(axis=1) > 40]
 .plot
 .bar(figsize=(16,9))
 .set(xlabel='Entity type',
      ylabel='Number of companies')
);

- 5510 Hotels and similar accommodation
- 4120 Construction of residential and non-residential buildings
- 7022 Business and other management consultancy activities
- 4941 Freight transport by road
- 4321 Electrical installation
- 7112 Engineering activities and related technical consultancy



### Remarks

- 5510 represents activity of cluster 3.
- Clusters 1 and 0 represent activity 4120 mainly.
- Cluster 0 represents 4321 and 4941.

### Activity type with respect to clusters (rate).

In [ ]:
# test
(df_final
 .assign(cluster=gmm.predict(df_final),
         cluster_count=lambda df:  df.groupby('cluster').transform('count').iloc[:, 0])
 
 .join(df_company_qualitative)
 .groupby(['cluster', 'FIRST_ACTIVITY_TYPE_ID', 'cluster_count'], as_index=False)
 .count()
 .dropna()
 
 .assign(rate=lambda df: df.ASSET.div(df.cluster_count))
 
 .set_index(['cluster', 'FIRST_ACTIVITY_TYPE_ID'])
 
 .rate
 
 .unstack(0)
 .dropna()
 .loc[lambda x: x.min(axis=1) > 0.003]
 .plot
 .bar(figsize=(16,9))
 .set(xlabel='Entity type',
      ylabel='Number of companies')
)

### Entity type with respect to clusters.

In [ ]:
(df_final
 .assign(cluster=gmm.predict(df_final))
 
 .join(df_company_qualitative)
 .groupby(['cluster', 'ENTITY_DETAILED_TYPE_LABEL'])
 .count()
 .ASSET
 .unstack(0)
 .dropna()
 .loc[lambda x: x.min(axis=1) > 40]
 .plot
 .bar(figsize=(16,9))
 .set(xlabel='Entity type',
      ylabel='Number of companies'));

### Remarks

- Cluster 3 contains the most of Sole Corporations and Joint-Stock companies.
- Cluster 0 contains the most LLC entities.
- Clusters are quite well balanced.

In [ ]:
(df_final
 .assign(cluster=gmm.predict(df_final))
 
 .join(df_company_adress)
 .groupby(['cluster', 'ENTITY_CITY'])
 .count()
 .ASSET
 .unstack(0)
 .dropna()
 .loc[lambda x: x.min(axis=1) > 40]
 .plot
 .bar(figsize=(16,9))
 .set(xlabel='Entity type',
      ylabel='Number of companies'));

### Remarks

- Most of the companies come from Madrid and Barcelona.
- Clusters are not discriminated by city.

## Cluster transition

In [ ]:
df = df_all.stack(1)
df.index = df.index.set_names(['year', 'id'])

In [ ]:
# Some values
df.sample(5)

In [ ]:
(df
 .join(df_company_quant[['ENTITY_START_DATE', 'NB_EMPLOYEES_2018']],
       on='id')
 .assign(AGE=lambda df: (2020 - df.ENTITY_START_DATE.dt.year))
 .drop(columns='ENTITY_START_DATE')
 .rename(columns={'NB_EMPLOYEES_2018': 'NB_EMPLOYEES'})
 .dropna()
 .assign(cluster=lambda df: gmm.predict(df))
 .cluster
 .unstack(0)
 .dropna()
 .head(25)
)

### Pickelize DataFrame

In [ ]:
(df
 .join(df_company_quant[['ENTITY_START_DATE', 'NB_EMPLOYEES_2018']],
       on='id')
 .assign(AGE=lambda df: (2020 - df.ENTITY_START_DATE.dt.year))
 .drop(columns='ENTITY_START_DATE')
 .rename(columns={'NB_EMPLOYEES_2018': 'NB_EMPLOYEES'})
 .dropna()
 .assign(cluster=lambda df: gmm.predict(df))
 .replace({'cluster': {i: cluster_names[i] for i in range(len(cluster_names))}})
 .to_pickle(path=path_data + 'df_transitions.pickle', protocol=4))

### Transitions statistics.

In [ ]:
(df
 .join(df_company_quant[['ENTITY_START_DATE', 'NB_EMPLOYEES_2018']],
       on='id')
 .assign(AGE=lambda df: (2020 - df.ENTITY_START_DATE.dt.year))
 .drop(columns='ENTITY_START_DATE')
 .rename(columns={'NB_EMPLOYEES_2018': 'NB_EMPLOYEES'})
 .dropna()
 .assign(cluster=lambda df: gmm.predict(df))
 .cluster
 .unstack(0)
 .dropna()
 .assign(transition_crisis=lambda df: df['2006'] == df['2007'])
 .transition_crisis
 .value_counts()
 .plot
 .bar()
)

In [ ]:
(df
 .join(df_company_quant[['ENTITY_START_DATE', 'NB_EMPLOYEES_2018']],
       on='id')
 .assign(AGE=lambda df: (2020 - df.ENTITY_START_DATE.dt.year))
 .drop(columns='ENTITY_START_DATE')
 .rename(columns={'NB_EMPLOYEES_2018': 'NB_EMPLOYEES'})
 .dropna()
 .assign(cluster=lambda df: gmm.predict(df))
 .cluster
 .unstack(0)
 .dropna()
 .loc[:, ['2009', '2010']]
 .astype(int)
 .apply(tuple, axis=1)
 .value_counts()
 .plot
 .bar(figsize=(16, 9))
)

In [ ]:
period = ['2007', '2008']

(df
 .join(df_company_quant[['ENTITY_START_DATE', 'NB_EMPLOYEES_2018']],
       on='id')
 .assign(AGE=lambda df: (2020 - df.ENTITY_START_DATE.dt.year))
 .drop(columns='ENTITY_START_DATE')
 .rename(columns={'NB_EMPLOYEES_2018': 'NB_EMPLOYEES'})
 .dropna()
 .assign(cluster=lambda df: gmm.predict(df))
 .cluster
 .unstack(0)
 .dropna()
 .loc[:, period]
 .astype(int)
 .groupby(period)
 .size()
 .unstack(0)
)

In [ ]:
period = ['2007', '2008']

data = (df
 .join(df_company_quant[['ENTITY_START_DATE', 'NB_EMPLOYEES_2018']],
       on='id')
 .assign(AGE=lambda df: (2020 - df.ENTITY_START_DATE.dt.year))
 .drop(columns='ENTITY_START_DATE')
 .rename(columns={'NB_EMPLOYEES_2018': 'NB_EMPLOYEES'})
 .dropna()
 .assign(cluster=lambda df: gmm.predict(df))
 .cluster
 .unstack(0)
 .dropna()
 .loc[:, period]
 .astype(int)
 .groupby(period)
 .size()
 .unstack(0)
)

In [ ]:
import plotly.express as px
# data=[[1, 25, 30, 50, 1], [20, 1, 60, 80, 30], [30, 60, 1, 5, 20]]
fig = px.imshow(data,
                labels=dict(x="New Cluster", y="Initial Cluster", color="Number of companies"),
                x=data.columns,
                y=data.columns,
                title="Clusters transition from 2007 to 2008")

fig.update_xaxes(side="top")
fig.show()

### Remarks

- Most of the companies of cluster 3 stay in cluster 3.
- Same for clusters 2 and 1.
- The principal transition is from cluster 3 to 2.
- Then it is from to to 3 and from 1 to 3.

### Experimentations on new rules of affectations

In [ ]:
(df
 .join(df_company_quant[['ENTITY_START_DATE', 'NB_EMPLOYEES_2018']],
       on='id')
 .assign(AGE=lambda df: (2020 - df.ENTITY_START_DATE.dt.year))
 .drop(columns='ENTITY_START_DATE')
 .rename(columns={'NB_EMPLOYEES_2018': 'NB_EMPLOYEES'})
 .dropna()
 .assign(cluster=lambda df: gmm.predict(df))
 .cluster
 .unstack(0)
 .dropna()
 .assign(visited=lambda df: (df.loc[:, pd.date_range('2001', '2009', freq= 'y').year.astype(str)]
                            .apply(set, axis=1)))
 .assign(test=lambda df: df['2009'].isin(df.visited))
 .assign(new_cluster=lambda df: df.apply(lambda df: df['2009'] not in df.visited, axis=1))
)

In [ ]:
pd.date_range('2001', '2009', freq= 'y').year.astype(str)